In [1]:
import pandas as pd
from clickhouse_driver import Client

host = "localhost"
port = 9099
user = "sjjc"
password = "taosan506"
database = "dm"
send_receive_timeout = 10
client = Client(host=host, port=port, user=user, password=password, database=database,
                send_receive_timeout=send_receive_timeout)

In [2]:
# 用户基本信息处理，选出所有的列来
sql = """
select uid from dm.pri_cust_base_info order by uid
"""
a = client.execute(sql)

In [3]:
name_attribute = ['uid']
data = pd.DataFrame(columns=name_attribute, data=a)
data.set_index('uid', inplace=True)

In [4]:
# 持有贷记卡张数
data_len = len(data)

In [5]:
def exec_eql_and_concat(data, sql, attr_list, default_na):
    # 执行sql
    data_list = client.execute(sql)
    # 转为df
    new_data_df = pd.DataFrame(columns=attr_list, data=data_list)
    # 按uid拼接
    new_data_df.set_index('uid', inplace=True)
    data1 = pd.merge(data,new_data_df,how='left',left_index=True, right_index = True,sort=False,copy=True)
    # data1 = data.join(new_data_df, how='left',left_index=True, right_index = True, sort=False)
    # 补全空缺值
    data1.fillna(default_na, inplace=True)
    # 返回
    return data1

In [6]:
select_djk_num = """
select uid, count(card_no) from dm.dm_v_as_djk_info group by uid order by uid;
"""
data = exec_eql_and_concat(data, select_djk_num, ['uid', 'djk_num'], {'djk_num': 0})

In [7]:
normal_djk_sql = """
select uid, count(card_no) from dm.dm_v_as_djk_info where card_sts_name='正常' group by uid order by uid
"""
data = exec_eql_and_concat(data, normal_djk_sql, ['uid', 'normal_djk_num'], {'normal_djk_num': 0})

In [8]:
# 用户逾期/止付卡片
abnormal_djk_num_sql = """
select uid, count(card_no) from dm.dm_v_as_djk_info where card_sts_name='银行止付卡片' or card_sts_name='因逾期停息停滞纳金' or card_sts_name='因逾期而停用卡片' group by uid order by uid
"""
data = exec_eql_and_concat(data, abnormal_djk_num_sql, ['uid', 'abnormal_djk_num'], {'abnormal_djk_num': 0})

In [9]:
not_using_djk_sql = """
select uid,1 from dm.dm_v_as_djk_info where card_sts_name!='正常' and uid not in (select uid from dm.dm_v_as_djk_info where card_sts_name='正常') group by uid
"""
data = exec_eql_and_concat(data, not_using_djk_sql, ['uid', 'not_using_djk'], {'not_using_djk': 0})

In [10]:
djk_credit_sum_sql = """
select uid, sum(cred_limit) from dm.dm_v_as_djk_info where card_sts_name='正常' group by uid order by uid
"""
data = exec_eql_and_concat(data, djk_credit_sum_sql, ['uid', 'djk_credit'], {'djk_credit': 0})

In [11]:
djk_deposit_sum_sql = """
select uid, sum(deposit) from dm.dm_v_as_djk_info where card_sts_name='正常' group by uid order by uid
"""
data = exec_eql_and_concat(data, djk_deposit_sum_sql, ['uid', 'djk_deposit'], {'djk_deposit': 0})

In [12]:
djk_over_draft_sum_sql = """
select uid, sum(over_draft) from dm.dm_v_as_djk_info group by uid order by uid
"""
data = exec_eql_and_concat(data, djk_over_draft_sum_sql, ['uid', 'djk_over_draft'], {'djk_over_draft': 0})

In [13]:
djk_dlay_amt_sum_sql = """
select uid, sum(dlay_amt) from dm.dm_v_as_djk_info group by uid order by uid
"""
data = exec_eql_and_concat(data, djk_dlay_amt_sum_sql, ['uid', 'djk_dlay_amt'], {'djk_dlay_amt': 0})

In [14]:
# 绑定还款账号数量
djk_bankacct_num_sql = """
select uid,count(bankacct) c from (
    select distinct bankacct,uid,bankacct_bal
    from dm_v_as_djk_info
    where bankacct!='""')
group by uid order by uid;
"""
data = exec_eql_and_concat(data, djk_bankacct_num_sql, ['uid', 'djk_bankacct_num'], {'djk_bankacct_num': 0})

In [15]:
# 绑定还款账号中总金额
djk_bankacct_sum_sql = """
select uid, sum(bankacct_bal) s
from (
    select distinct bankacct,uid,bankacct_bal
    from dm_v_as_djk_info
    where bankacct!='""')
group by uid order by uid
"""
data = exec_eql_and_concat(data, djk_bankacct_sum_sql, ['uid', 'djk_bankacct_sum'], {'djk_bankacct_sum': 0})

In [16]:
# 绑定了etc
djk_is_etc_sql = """
select uid,1 from dm.dm_v_as_djk_info where uid in (select uid from dm.dm_v_as_djk_info where is_etc='Y') group by uid;
"""
data = exec_eql_and_concat(data, djk_is_etc_sql, ['uid', 'djk_is_etc'], {'djk_is_etc': 0})

In [17]:
# 渠道属于网络
djk_mode_online_sql = """
select uid,1 from dm.dm_v_as_djk_info where uid  in (select uid from dm.dm_v_as_djk_info where issue_mode_name!='') group by uid;
"""
data = exec_eql_and_concat(data, djk_mode_online_sql, ['uid', 'djk_mode_online'], {'djk_mode_online': 0})

In [18]:
# 历史总计逾期期数
djk_sum_dlay_mths_sql = """
select uid, sum(dlay_mths) from dm.dm_v_as_djk_info group by uid order by uid
"""
data = exec_eql_and_concat(data, djk_sum_dlay_mths_sql, ['uid', 'djk_sum_dlay_mths'], {'djk_sum_dlay_mths': 0})

以上完成了对djk_info表的特征提取
如下进行djkfq表的特征提取

In [19]:
# 分期付款总金额
djkfq_total_amt_sql = """
select uid, sum(total_amt) from dm_v_as_djkfq_info group by uid order by uid
"""
data = exec_eql_and_concat(data, djkfq_total_amt_sql, ['uid', 'djkfq_total_amt'], {'djkfq_total_amt': 0})

In [20]:
# 分期付款总笔数
djkfq_total_num_sql = """
select uid, count(mp_number) from dm_v_as_djkfq_info group by uid order by uid
"""
data = exec_eql_and_concat(data, djkfq_total_num_sql, ['uid', 'djkfq_total_num'], {'djkfq_total_num': 0})

In [21]:
# 平均分期付款月数
djkfq_avg_mths_sql = """
select uid, avg(total_mths) from dm_v_as_djkfq_info group by uid order by uid
"""
data = exec_eql_and_concat(data, djkfq_avg_mths_sql, ['uid', 'djkfq_avg_mths'], {'djkfq_avg_mths': 0})

In [22]:
# 每月摊销付款金额
djkfq_sum_mth_instl_sql = """
select uid, sum(mth_instl) from dm_v_as_djkfq_info group by uid order by uid;
"""
data = exec_eql_and_concat(data, djkfq_sum_mth_instl_sql, ['uid', 'djkfq_sum_mth_instl'], {'djkfq_sum_mth_instl': 0})

In [23]:
# 剩余未还本金+费用总和
djkfq_remain_unpay_money_sql = """
select uid, sum(rem_ppl+rem_fee) from dm_v_as_djkfq_info group by uid order by uid
"""
data = exec_eql_and_concat(data, djkfq_remain_unpay_money_sql, ['uid', 'djkfq_remain_unpay_money'],
                           {'djkfq_remain_unpay_money': 0})

In [24]:
# 分析contract表
contract_01_sql = """
select uid,count(*) c,
       sum(buss_amt) ,
       sum(norm_bal),
       sum(dlay_bal),
       sum(dull_bal),
       sum(frz_amt)
from dm_v_tr_contract_mx
group by uid order by uid;
"""

data = exec_eql_and_concat(data,
                           contract_01_sql,
                           ['uid',
                            'contract_mx_num',
                            'contract_mx_buss_amt',
                            'contract_mx_norm_bal',
                            'contract_mx_dlay_bal',
                            'contract_mx_dull_bal',
                            'contract_mx_frz_amt'
                            ], {'contract_mx_num': 0,
                                'contract_mx_buss_amt': 0,
                                'contract_mx_norm_bal': 0,
                                'contract_mx_dlay_bal': 0,
                                'contract_mx_dull_bal': 0,
                                'contract_mx_frz_amt': 0})

In [25]:
# contract 是否买车
contract_buy_car_sql = """
select uid,1
from dm.dm_v_tr_contract_mx
where  uid  in (select uid from dm.dm_v_tr_contract_mx where loan_use like '%车%') group by uid
;
"""

data = exec_eql_and_concat(data,
                           contract_buy_car_sql,
                           ['uid','contract_buy_car'],
                           {'contract_buy_car': 0})

In [26]:
# contract 是否买房
contract_buy_house_sql = """
select uid,1
from dm.dm_v_tr_contract_mx
where uid  in (select uid from dm.dm_v_tr_contract_mx where loan_use like '%房%') group by uid
;
"""

data = exec_eql_and_concat(data,
                           contract_buy_house_sql,
                           ['uid','contract_buy_house'],
                           {'contract_buy_house': 0})

In [27]:
# contract 是否贷款经营
contract_business_sql = """
select uid,1
from dm.dm_v_tr_contract_mx
where uid  in (select uid from dm.dm_v_tr_contract_mx where loan_use like '%经营%') group by uid
;
"""

data = exec_eql_and_concat(data,
                           contract_business_sql,
                           ['uid','contract_business'],
                           {'contract_business': 0})

In [28]:
# contract 是否贷款消费
contract_consume_sql = """
select uid,1
from dm.dm_v_tr_contract_mx
where uid  in (select uid from dm.dm_v_tr_contract_mx where loan_use like '%消费%') group by uid
;
"""

data = exec_eql_and_concat(data,
                           contract_consume_sql,
                           ['uid','contract_consume'],
                           {'contract_consume': 0})

In [29]:
# contract 买房/车/经营/消费总金额,交易次数,avg期数
contract_buy_car_sum_info_sql = """
select uid,
sum(buss_amt),
count(*),
avg(term_mth)
from dm_v_tr_contract_mx where loan_use like  '%车%' group by uid;
"""

data = exec_eql_and_concat(data,
                           contract_buy_car_sum_info_sql,
                           ['uid','contract_buy_car_sum','contract_buy_car_time','contract_buy_car_term_month'],
                           {'contract_buy_car_sum': 0,
                            'contract_buy_car_time':0,
                            'contract_buy_car_term_month':0})

In [30]:

contract_buy_house_sum_info_sql = """
select uid,
sum(buss_amt),
count(*),
avg(term_mth)
from dm_v_tr_contract_mx where loan_use like  '%房%' group by uid;
"""

data = exec_eql_and_concat(data,
                           contract_buy_house_sum_info_sql,
                           ['uid','contract_buy_house_sum','contract_buy_house_time','contract_buy_house_term_month'],
                           {'contract_buy_house_sum': 0,
                            'contract_buy_house_time':0,
                            'contract_buy_house_term_month':0})

In [31]:

contract_business_sum_info_sql = """
select uid,
sum(buss_amt),
count(*),
avg(term_mth)
from dm_v_tr_contract_mx where loan_use like  '%经营%' group by uid;
"""

data = exec_eql_and_concat(data,
                           contract_business_sum_info_sql,
                           ['uid','contract_business_sum','contract_business_time','contract_business_term_month'],
                           {'contract_business_sum': 0,
                            'contract_business_time':0,
                            'contract_business_term_month':0})

In [32]:

contract_consume_sum_info_sql = """
select uid,
sum(buss_amt),
count(*),
avg(term_mth)
from dm_v_tr_contract_mx where loan_use like  '%消费%' group by uid;
"""

data = exec_eql_and_concat(data,
                           contract_consume_sum_info_sql,
                           ['uid','contract_consume_sum','contract_consume_time','contract_consume_term_month'],
                           {'contract_consume_sum': 0,
                            'contract_consume_time':0,
                            'contract_consume_term_month':0})

下面分析贷记卡数据

In [33]:
djk_mx_info_sql = """
select uid, count(*),sum(tran_amt),avg(tran_amt),max(tran_amt)
from dm.dm_v_tr_djk_mx group by uid order by uid;
"""
data = exec_eql_and_concat(data,
                           djk_mx_info_sql,
                           ['uid','djk_mx_count','djk_mx_sum','djk_mx_avg','djk_mx_max'],
                           {'djk_mx_count': 0,
                            'djk_mx_sum':0,
                            'djk_mx_avg':0,
                            'djk_mx_max':0})

In [34]:
# 贷记卡支出数据+
djk_mx_expense_sql = """
select uid, count(*),sum(tran_amt),avg(tran_amt),max(tran_amt)
from dm.dm_v_tr_djk_mx where tran_amt_sign='+' group by uid order by uid;
"""
data = exec_eql_and_concat(data,
                           djk_mx_expense_sql,
                           ['uid','djk_mx_expense_count','djk_mx_expense_sum','djk_mx_expense_avg','djk_mx_expense_max'],
                           {'djk_mx_expense_count': 0,
                            'djk_mx_expense_sum':0,
                            'djk_mx_expense_avg':0,
                            'djk_mx_expense_max':0})

In [35]:
# 贷记卡收入数据-
djk_mx_income_sql = """
select uid, count(*),sum(tran_amt),avg(tran_amt),max(tran_amt)
from dm.dm_v_tr_djk_mx where tran_amt_sign='-' group by uid order by uid;
"""
data = exec_eql_and_concat(data,
                           djk_mx_income_sql,
                           ['uid','djk_mx_income_count','djk_mx_income_sum','djk_mx_income_avg','djk_mx_income_max'],
                           {'djk_mx_income_count': 0,
                            'djk_mx_income_sum':0,
                            'djk_mx_income_avg':0,
                            'djk_mx_income_max':0})

In [36]:
# 净值变化
djk_mx_realchange_sql = """
select uid,sum(if(tran_amt_sign='+',1,-1)*tran_amt) from dm.dm_v_tr_djk_mx group by uid order by uid
"""
data = exec_eql_and_concat(data,
                           djk_mx_realchange_sql,
                           ['uid','djk_mx_real_tran_amt'],
                           {'djk_mx_real_tran_amt': 0})

In [37]:
dsf_mx_info_sql = """
select uid, count(*), sum(tran_amt), avg(tran_amt), max(tran_amt) from dm.dm_v_tr_dsf_mx group by uid order by uid
"""
data = exec_eql_and_concat(data,
                           dsf_mx_info_sql,
                           ['uid','dsf_mx_count','dsf_mx_sum','dsf_mx_avg','dsf_mx_max'],
                           {'djk_mx_real_tran_amt': 0,
                            'dsf_mx_sum': 0,'dsf_mx_avg': 0,'dsf_mx_max': 0})

In [38]:
data.to_csv('./result1-5.csv', encoding='utf-8')